<br>
<h1 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">PyTorch Tutorial Series<br> Inference Series Part I</h1>
<br>

**VERSION 52: SETI - To check code on SETI, do go to that version or before.**

**VERSION 53 and beyond-: Petfinder**

# NOTE: THERE ARE SOME CODE CHANGES IN INFERENCE ALL FOLDS, REFER TO MY SOURCE CODE AND CHANGE THE NARRATIVE ACCORDINGLY.
# NOTE: THERE ARE SOME CODE CHANGES IN INFERENCE ALL FOLDS, REFER TO MY SOURCE CODE AND CHANGE THE NARRATIVE ACCORDINGLY.
# NOTE: THERE ARE SOME CODE CHANGES IN INFERENCE ALL FOLDS, REFER TO MY SOURCE CODE AND CHANGE THE NARRATIVE ACCORDINGLY.

This notebook is a **GENERIC NOTEBOOK** talking about PyTorch's saving and loading, plus some details of inference I learnt during my learning journey. During each new competition I embark on, I will always improve it.

---

This notebook is part I of the PyTorch Tutorial Inference Series. I will detail on how to save and load weights in PyTorch. I will split this notebook into two parts:

1. First part: The real inference in action - for now, I will just add SETI as the main competition. However, I intend to share this notebook across multiple different competitions.

2. The tutorial on PyTorch.

---

I have added a back to top button for each section for easy navigation.

References:

1. [Using args and kwargs](https://note.nkmk.me/en/python-args-kwargs-usage/#:~:text=In%20Python%2C%20by%20adding%20*%20and,arguments)

<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background:black; border:0' role="tab" aria-controls="home"><center>Quick Navigation</center></h3>

    
* [Dependencies](#1)
    * [Import modules offline](#Import_modules_offline)
    * [Import other modules](#Import_other_modules)
* [Configurations](#2)
* [Utility](#30)
    * [Seeding](#3)
    * [Numpy to Latex](#31)
* [Prepare Data](#Prepare_Data)
* [Dataset](#Dataset)
* [Augmentations](#Augmentations)
* [Model Instantiation](#Model)    
* [Inference by Folds](#8)
* [Post-Processing Techniques](#PP)
    * [Forward Ensembling]
    * [Training Image Embeddings]
* [Saving and Loading Model Weights](#20)    
* [Dissecting Inference by Folds](#60)
    * [Problem Settings](#61)

<a id="1"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Dependencies</h1>

### Import modules offline <a id="Import_modules_offline"></a>

Most Kaggle competitions need you to inference in **offline** mode. 

- Firstly, download the `github` repo as a zip file.
- Then, upload the zip file/folder as a Kaggle dataset and add the dataset to your Kaggle notebook.
- Lastly, use `sys.path.append()` to put the path correctly in your environment. See example below.

In [ ]:
import sys

timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
pytorch_gradcam_path = "../input/reighns-gradcam/pytorch-grad-cam-master"
pytorch_ttach_path = "../input/reighns-ttach/ttach-master"

sys_paths = [timm_path, pytorch_gradcam_path, pytorch_ttach_path]

for paths in sys_paths:
    sys.path.append(paths)

import timm

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

### Import other modules <a id="Import_other_modules"></a>

We import the other modules normally since they are in Kaggle's environment already.

In [ ]:
import collections
import gc
import math
import os
import random
import warnings
from collections import defaultdict
from dataclasses import asdict, dataclass, field
from typing import *

import albumentations
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import timm
import torch
import torch.nn.functional as F
from albumentations.pytorch.transforms import ToTensorV2
from matplotlib import pyplot as plt
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.autograd import Variable
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.optimizer import Optimizer
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")
import collections
import gc
import logging
import os
import pathlib
import random
# import torchsummary
import warnings
from collections import defaultdict
from dataclasses import asdict, dataclass, field
from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger
from pathlib import Path
from typing import *
from typing import Any, Dict, List

import numpy as np
import pandas as pd
import timm
import torch
from IPython.display import clear_output
from torch.optim.lr_scheduler import (CosineAnnealingLR,
                                      CosineAnnealingWarmRestarts,
                                      ReduceLROnPlateau)
from torch.utils.data import DataLoader, Dataset



<a href="#top">Back to top</a>

<a id="2"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Configurations</h1>

### Config

At least for me, config is the most important part for my pipelines. It is also a good practice to have a configuration file to keep track of experiments.

You will almost always see Kagglers have a `config` class or a dict so that they can call the parameters. To go one step deeper, you can dump the dict into your favourite MLOps platform (read: wandb!).

In [ ]:
from pathlib import Path
import pathlib

class config:
    warnings.filterwarnings("ignore", category=UserWarning)
    
    DATA_DIR = Path("../input/petfinder-pawpularity-score")
    OUTPUT_DIR = Path("./")
    LOGS_DIR = Path(OUTPUT_DIR, "logs")
    MODEL_DIR = Path(OUTPUT_DIR, "model")
    OOF_DIR = Path(OUTPUT_DIR, "oof")
    LOGS_DIR.mkdir(parents=True, exist_ok=True)
    MODEL_DIR.mkdir(parents=True, exist_ok=True)
    OOF_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
def init_logger(log_file: str = Path(config.LOGS_DIR, "info.log")) -> logging.Logger:
    """Initialize logger and save to file.

    Consider having more log_file paths to save, eg: debug.log, error.log, etc.

    Args:
        log_file (str, optional): [description]. Defaults to Path(LOGS_DIR, "info.log").

    Returns:
        logging.Logger: [description]
    """
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    stream_handler = StreamHandler()
    stream_handler.setFormatter(
        Formatter("%(asctime)s: %(message)s", "%Y-%m-%d %H:%M:%S")
    )
    file_handler = FileHandler(filename=log_file)
    file_handler.setFormatter(
        Formatter("%(asctime)s: %(message)s", "%Y-%m-%d %H:%M:%S")
    )
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)

    return logger

In [ ]:
config.logger = init_logger()

In [ ]:
from dataclasses import dataclass, field, asdict
import pathlib
from typing import Any, Dict, List


@dataclass
class FilePaths:
    """Class to keep track of the files."""

    train_images: pathlib.Path = pathlib.Path(config.DATA_DIR, "train")
    test_images: pathlib.Path = pathlib.Path(config.DATA_DIR, "test")
    train_csv: pathlib.Path = pathlib.Path(config.DATA_DIR, "train.csv")
    test_csv: pathlib.Path = pathlib.Path(config.DATA_DIR, "test.csv")
    sub_csv: pathlib.Path = pathlib.Path(
        config.DATA_DIR, "sample_submission.csv"
    )
    folds_csv: pathlib.Path = pathlib.Path(
        config.OUTPUT_DIR, "df_folds.csv"
    )
    weight_path: pathlib.Path = pathlib.Path(config.MODEL_DIR)
    oof_csv: pathlib.Path = pathlib.Path(config.OOF_DIR)

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary."""
        return asdict(self)


@dataclass
class DataLoaderParams:
    """Class to keep track of the data loader parameters."""

    train_loader: Dict[str, Any] = field(
        default_factory=lambda: {
            "batch_size": 32,
            "num_workers": 0,
            "pin_memory": False,
            "drop_last": True,
            "shuffle": True,
            "collate_fn": None,
        }
    )
    valid_loader: Dict[str, Any] = field(
        default_factory=lambda: {
            "batch_size": 32,
            "num_workers": 0,
            "pin_memory": False,
            "drop_last": False,
            "shuffle": False,
            "collate_fn": None,
        }
    )

    test_loader: Dict[str, Any] = field(
        default_factory=lambda: {
            "batch_size": 8,
            "num_workers": 0,
            "pin_memory": True,
            "drop_last": False,
            "shuffle": False,
            "collate_fn": None,
        }
    )

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary."""
        return asdict(self)
    
    def get_len_train_loader(self):
        total_rows = pd.read_csv(FilePaths().train_csv).shape[0] # get total number of rows/images
        total_rows_per_fold = total_rows / (MakeFolds().num_folds)
        total_rows_per_training = total_rows_per_fold * (MakeFolds().num_folds - 1) # if got 1000 images, 10 folds, then train on 9 folds = 1000/10 * (10-1) = 100 * 9 = 900
        len_of_train_loader = total_rows_per_training // self.train_loader["batch_size"] # if 900 rows, bs is 16, then 900/16 = 56.25, but we drop last if dataloader, so become 56 steps. if not 57 steps.
        return int(len_of_train_loader)

@dataclass
class MakeFolds:
    """A class to keep track of cross-validation schema.
    seed (int): random seed for reproducibility.
    num_folds (int): number of folds.
    cv_schema (str): cross-validation schema.
    class_col_name (str): name of the target column.
    image_col_name (str): name of the image column.
    folds_csv (str): path to the folds csv.
    """

    seed: int = 1992
    num_folds: int = 5
    cv_schema: str = "StratifiedKFold"
    class_col_name: str = "Pawpularity"
    image_col_name: str = "Id"
    folds_csv: pathlib.Path = FilePaths().folds_csv

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary."""
        return asdict(self)


@dataclass
class AugmentationParams:
    """Class to keep track of the augmentation parameters."""

    mean: List[float] = field(default_factory=lambda: [0.485, 0.456, 0.406])
    std: List[float] = field(default_factory=lambda: [0.229, 0.224, 0.225])
    image_size: int = 224
    mixup: bool = True
    mixup_params: Dict[str, Any] = field(
        default_factory=lambda: {"mixup_alpha": 0.5, "use_cuda": True}
    )

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary."""
        return asdict(self)

@dataclass
class CriterionParams:
    """A class to track loss function parameters."""

    train_criterion_name: str = "BCEWithLogitsLoss"
    valid_criterion_name: str = "BCEWithLogitsLoss"
    train_criterion_params: Dict[str, Any] = field(
        default_factory=lambda: {
            "weight": None,
            "reduction": "mean",
            "pos_weight": None,
        }
    )
    valid_criterion_params: Dict[str, Any] = field(
        default_factory=lambda: {
            "weight": None,
            "reduction": "mean",
            "pos_weight": None,
        }
    )
@dataclass
class OptimizerParams:
    """A class to track optimizer parameters.
    optimizer_name (str): name of the optimizer.
    lr (float): learning rate.
    weight_decay (float): weight decay.
    """

    optimizer_name: str = "AdamW"
    optimizer_params: Dict[str, Any] = field(
        default_factory=lambda: {
            "lr": 5e-5,
            "betas": (0.9, 0.999),
            "amsgrad": False,
            "weight_decay": 1e-3,
            "eps": 1e-08,
        }
    )

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary."""
        return asdict(self)

        
        return torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-5, 
                                                   steps_per_epoch=len(train_loader), epochs=6)
@dataclass
class SchedulerParams:
    """A class to track Scheduler Params."""

    scheduler_name: str = "OneCycleLR"
    if scheduler_name == "OneCycleLR":
        scheduler_params: Dict[str, Any] = field(
            default_factory=lambda: {
                "max_lr": 3e-5,
                "steps_per_epoch": DataLoaderParams().get_len_train_loader(),
                "epochs": 6,
                "last_epoch": -1,
            }
        )

    elif scheduler_name == "CosineAnnealingWarmRestarts":
        scheduler_params: Dict[str, Any] = field(
            default_factory=lambda: {
                "T_0": 10,
                "T_mult": 1,
                "eta_min": 1e-4,
                "last_epoch": -1,
            }
        )

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary."""
        return asdict(self)
    

        
@dataclass
class ModelParams:
    """A class to track model parameters.
    model_name (str): name of the model.
    pretrained (bool): If True, use pretrained model.
    input_channels (int): RGB image - 3 channels or Grayscale 1 channel
    output_dimension: Final output neuron.
                      It is the number of classes in classification.
                      Caution: If you use sigmoid layer for Binary, then it is 1.
    """

    # model_name: str = "tf_efficientnet_b4_ns"
    model_name: str = "swin_large_patch4_window7_224"
    pretrained: bool = True
    input_channels: int = 3
    output_dimension: int = 1 

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary."""
        return asdict(self)
    
    def check_dimension(self):
        if CriterionParams().criterion_name == "BCEWithLogitsLoss":
            assert output_dimension == 1, "This loss uses sigmoid layer, so expect output to be 1."



@dataclass
class GlobalTrainParams:
    debug: bool = False
    debug_multipler: int = 16
    epochs: int = 6 # 6
    mixup: bool = AugmentationParams().mixup
    model_name: str = ModelParams().model_name
    num_classes: int = ModelParams().output_dimension
    classification_type: str = "multiclass"
    use_amp: bool = True
        
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary."""
        return asdict(self)
    
    
FILES = FilePaths()
FOLDS = MakeFolds()
LOADER_PARAMS = DataLoaderParams()
TRAIN_PARAMS = GlobalTrainParams()
TRANSFORMS = AugmentationParams()
CRITERION_PARAMS = CriterionParams()
OPTIMIZER_PARAMS = OptimizerParams()
SCHEDULER_PARAMS = SchedulerParams()
MODEL_PARAMS = ModelParams()

<a href="#top">Back to top</a>

<a id="3"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Utils</h1>

### Seeding and Reproducibility

This is very important, and I tend to have an OCD over it. Everytime I made changes to my code base, I will run the whole script on DEBUG mode just to check if the training loss changes. If my code change should not affect the training results, yet the loss changes, then it either means that my code is wrong, or a hidden pseudo number generator is called.

In [ ]:
def seed_all(seed: int = 1992):
    """Seed all random number generators."""
    print("Using Seed Number {}".format(seed))

    os.environ["PYTHONHASHSEED"] = str(
        seed
    )  # set PYTHONHASHSEED env var at fixed value
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)  # pytorch (both CPU and CUDA)
    np.random.seed(seed)  # for numpy pseudo-random generator
    # set fixed value for python built-in pseudo-random generator
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True


def seed_worker(_worker_id):
    """Seed a worker with the given ID."""
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

seed_all()

<a href="#top">Back to top</a>

### Converting Numpy Arrays into Latex Form

In [ ]:
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError("bmatrix can at most display two dimensions")
    lines = (
        np.array2string(a, max_line_width=np.infty)
        .replace("[", "")
        .replace("]", "")
        .splitlines()
    )
    rv = [r"\begin{bmatrix}"]
    rv += ["  " + " & ".join(l.split()) + r"\\" for l in lines]
    rv += [r"\end{bmatrix}"]
    return "\n".join(rv)

<a href="#top">Back to top</a>

<a id="Prepare_Data"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Prepare Data</h1>

### Functions to prepare data, datasets and dataloaders

We won't go into details here as this is a inference-centric notebook.

In [ ]:
import os
from pathlib import Path
from typing import Union

import pandas as pd
import torch

def return_filepath(image_id: str, folder: Path = FILES.train_images) -> str:
    """Add a new column image_path to the train and test csv.
    We can call the images easily in __getitem__ in Dataset.
    We need to be careful if the image_id has extension already.
    In this case, there is no need to add the extension.
    Args:
        image_id (str): The unique image id: 1000015157.jpg
        folder (Path, optional): The train folder. Defaults to FILES().train_images.
    Returns:
        image_path (str): The path to the image: "c:\\users\\reighns\\kaggle_projects\\cassava\\data\\train\\1000015157.jpg"
    """
    image_path = os.path.join(folder, f"{image_id}.jpg")
    return image_path


def prepare_data(
    image_col_name: str = FOLDS.image_col_name,
) -> pd.DataFrame:
    """Call a sequential number of steps to prepare the data.
    Args:
        image_col_name (str): The column name of the unique image id.
                        In Cassava, it is "image_id".
    Returns:
        df_train (pd.DataFrame): The train dataframe.
        df_test (pd.DataFrame): The test dataframe.
        df_folds (pd.DataFrame): The folds dataframe with an additional column "fold".
        df_sub (pd.DataFrame): The submission dataframe.
    """

    df_train = pd.read_csv(FILES.train_csv)
    df_test = pd.read_csv(FILES.test_csv)
    df_sub = pd.read_csv(FILES.sub_csv)

    df_train["image_path"] = df_train[image_col_name].apply(
        lambda x: return_filepath(image_id=x, folder=FILES.train_images)
    )
    df_test["image_path"] = df_test[image_col_name].apply(
        lambda x: return_filepath(x, folder=FILES.test_images)
    )
    
    ##################### CUSTOM Sturges' rule ####################################
    num_bins = int(np.floor(1+np.log2(len(df_train))))
    # Cut the target Pawpularity into `num_bins` using pd.cut
    df_train['bins'] = pd.cut(df_train['Pawpularity'], bins=num_bins, labels=False, ordered=True)
    df_train['bins'].hist()
    ################################################################################

    df_folds = make_folds(train_csv=df_train, cv_params=FOLDS)

    return df_train, df_test, df_folds, df_sub


def prepare_loaders(
    df_folds: pd.DataFrame,
    fold: int,
) -> Union[torch.utils.data.DataLoader, torch.utils.data.DataLoader]:
    """Prepare Data Loaders."""

    if TRAIN_PARAMS.debug:
        df_train = df_folds[df_folds["fold"] != fold].sample(
            LOADER_PARAMS.train_loader["batch_size"]
            * TRAIN_PARAMS.debug_multipler, 
            random_state = FOLDS.seed
        )
        df_valid = df_folds[df_folds["fold"] == fold].sample(
            LOADER_PARAMS.train_loader["batch_size"]
            * TRAIN_PARAMS.debug_multipler, 
            random_state = FOLDS.seed
        )
        # TODO: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html
        # TODO: Consider adding stratified sampling to avoid df_valid having 0 num samples of minority class, causing issues when computing roc.
        df_oof = df_valid.copy()
    else:
        df_train = df_folds[df_folds["fold"] != fold].reset_index(drop=True)
        df_valid = df_folds[df_folds["fold"] == fold].reset_index(drop=True)
        # Initiate OOF dataframe for this fold (same as df_valid).
        df_oof = df_valid.copy()

    dataset_train = CustomDataset(
        df_train, transforms=get_train_transforms(), mode="train"
    )
    dataset_valid = CustomDataset(
        df_valid, transforms=get_valid_transforms(), mode="train"
    )

    # Seeding workers for reproducibility.
    g = torch.Generator()
    g.manual_seed(0)

    train_loader = torch.utils.data.DataLoader(
        dataset_train,
        **LOADER_PARAMS.train_loader,
        worker_init_fn=seed_worker,
        generator=g,
    )
    valid_loader = torch.utils.data.DataLoader(
        dataset_valid,
        **LOADER_PARAMS.valid_loader,
        worker_init_fn=seed_worker,
        generator=g,
    )

    # TODO: consider decoupling the oof and loaders, and consider add test loader here for consistency.
    return train_loader, valid_loader, df_oof

### Cross-Validation Strategy

We won't go into details here as this is a inference-centric notebook. But a snippet is here:

```python
if cv_params.cv_schema == "StratifiedKFold":
    df_folds = train_csv.copy()
    skf = StratifiedKFold(
        n_splits=cv_params.num_folds,
        shuffle=True,
        random_state=cv_params.seed,
    )
        df_folds.loc[val_idx, "fold"] = int(fold + 1)
    df_folds["fold"] = df_folds["fold"].astype(int)
    print(df_folds.groupby(["fold", cv_params.class_col_name]).size())     
```

In [ ]:
import pandas as pd

# from IPython.display import display
from sklearn.model_selection import GroupKFold, StratifiedKFold


def make_folds(
    train_csv: pd.DataFrame, cv_params: MakeFolds()
) -> pd.DataFrame:
    """Split the given dataframe into training folds."""

    if cv_params.cv_schema == "StratifiedKFold":
        df_folds = train_csv.copy()
        skf = StratifiedKFold(
            n_splits=cv_params.num_folds,
            shuffle=True,
            random_state=cv_params.seed,
        )
        ######################## Custom Stratification ############
        for fold, (_train_idx, val_idx) in enumerate(
            skf.split(
                X=df_folds[cv_params.image_col_name],
                y=df_folds['bins'],
            )
        ):
        ##########################################################
            df_folds.loc[val_idx, "fold"] = int(fold + 1)
        df_folds["fold"] = df_folds["fold"].astype(int)
        print(df_folds.groupby(["fold", cv_params.class_col_name]).size())

    elif cv_params.cv_schema == "GroupKfold":
        df_folds = train_csv.copy()
        gkf = GroupKFold(n_splits=cv_params.num_folds)
        groups = df_folds[cv_params.group_kfold_split].values
        for fold, (_train_index, val_index) in enumerate(
            gkf.split(
                X=df_folds, y=df_folds[cv_params.class_col_name], groups=groups
            )
        ):
            df_folds.loc[val_index, "fold"] = int(fold + 1)
        df_folds["fold"] = df_folds["fold"].astype(int)

        print(df_folds.groupby(["fold", cv_params.class_col_name]).size())

    else:
        config.logger.error(
            f"Unknown CV schema: {cv_params.cv_schema}, are you using custom split?"
        )
        df_folds = train_csv.copy()
        print(df_folds.groupby(["fold", cv_params.class_col_name]).size())

    df_folds.to_csv(cv_params.folds_csv, index=False)

    return df_folds

In [ ]:
df_train, df_test, df_folds, df_sub = prepare_data()
display(df_train.head())

In [ ]:
is_normalize_target = True

if is_normalize_target:
    df_folds["Pawpularity"] = df_folds["Pawpularity"] / 100

<a href="#top">Back to top</a>

<a id="Dataset"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Dataset</h1>

`Dataset` may be unique to each competition. But in general, they have a fixed framework as follows. Here is a snippet of what a typical `dataset` class looks like! Note, you need to expand the cell to see the full code!

```python
class CustomDataset(torch.utils.data.Dataset):
    def __len__(self) -> int:
        """Return the length of the dataset."""
        return len(self.df)
    
    def __getitem__(
        self, index: int
    ) -> Union[
        Dict[str, torch.FloatTensor],
        Dict[str, Union[torch.FloatTensor, torch.LongTensor]],
    ]:
        """Implements the getitem method: https://www.geeksforgeeks.org/__getitem__-and-__setitem__-in-python/

        Be careful of Targets:
            BCEWithLogitsLoss expects a target.float()
            CrossEntropyLoss expects a target.long()

        Args:
            index (int): index of the dataset.

        Returns:
            Dict[str, torch.FloatTensor]:{"X": image_tensor}
            Dict[str, Union[torch.FloatTensor, torch.LongTensor]]: {"y": target_tensor} If BCEwithLogitsLoss then FloatTensor, else LongTensor
        """
        image_path = self.image_path[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Get target for all modes except for test, if test, replace target with dummy ones to pass through return_dtype.
        target = self.targets[index] if self.mode != "test" else torch.ones(1)

        if self.transforms:
            image = self.transforms(image=image)["image"]

        X, y, original_image = self.return_dtype(image, target, original_image)
        
        if self.mode in ["train", "valid"]:
            return {"X": X, "y": y}

        if self.mode == "test":
            return {"X": X}
```

In [ ]:
from typing import Dict, Union
import albumentations

import cv2
import pandas as pd
import torch


class CustomDataset(torch.utils.data.Dataset):
    """Dataset class for the {insert competition/project name} dataset."""

    def __init__(
        self,
        df: pd.DataFrame,
        transforms: albumentations.core.composition.Compose = None,
        mode: str = "train",
    ):
        """Constructor for the dataset class.

        Args:
            df (pd.DataFrame): Dataframe for either train, valid or test.
            transforms (albumentations.core.composition.Compose): albumentations transforms to apply to the images.
            mode (str, optional): Defaults to "train". One of ['train', 'valid', 'test', 'gradcam']
        """

        # "image_path" is hardcoded, as that is always defined in prepare_data.
        self.image_path = df["image_path"].values
        self.image_ids = df[FOLDS.image_col_name].values
        self.df = df
        self.targets = (
            torch.from_numpy(df[FOLDS.class_col_name].values)
            if mode != "test"
            else None
        )

        self.transforms = transforms
        self.mode = mode

        if self.mode not in ["train", "valid", "test", "gradcam"]:
            raise ValueError(
                f"Mode {self.mode} not in accepted list of modes {['train', 'valid', 'test', 'gradcam']}"
            )

    def __len__(self) -> int:
        """Return the length of the dataset."""
        return len(self.df)

    @staticmethod
    def return_dtype(
        X: torch.Tensor, y: torch.Tensor, original_image: torch.Tensor
    ) -> torch.Tensor:
        """Return the dtype of the dataset.
        Args:
            X (torch.Tensor): Image tensor.
            y (torch.Tensor): Target tensor.
            original_image  (torch.Tensor): Original image tensor.
        Returns:
            X (torch.Tensor): Image tensor.
            y (torch.Tensor): Target tensor.
            original_image  (torch.Tensor): Original image tensor.
        """

        # TODO: To check if defining float32 here will affect how Mixed Precision works. If so, then change from float32 to float.
        if CRITERION_PARAMS.train_criterion_name == "BCEWithLogitsLoss":
            # Make changes to reshape rather than in Trainer.
            y = torch.as_tensor(y, dtype=torch.float32).view(-1, 1)
        else:
            y = torch.as_tensor(y, dtype=torch.long)

        X = torch.as_tensor(X, dtype=torch.float32)
        original_image = torch.as_tensor(original_image, dtype=torch.float32)

        return X, y, original_image
    

    def check_shape(self):
        """Check the shape of the dataset.

        Add a tensor transpose if transformation is None since most images is HWC but ToTensorV2 transforms them to CHW."""

        raise NotImplementedError

    def __getitem__(
        self, index: int
    ) -> Union[
        Dict[str, torch.FloatTensor],
        Dict[str, Union[torch.FloatTensor, torch.LongTensor]],
    ]:
        """Implements the getitem method: https://www.geeksforgeeks.org/__getitem__-and-__setitem__-in-python/

        Be careful of Targets:
            BCEWithLogitsLoss expects a target.float()
            CrossEntropyLoss expects a target.long()

        Args:
            index (int): index of the dataset.

        Returns:
            Dict[str, torch.FloatTensor]:{"X": image_tensor}
            Dict[str, Union[torch.FloatTensor, torch.LongTensor]]: {"y": target_tensor} If BCEwithLogitsLoss then FloatTensor, else LongTensor
        """
        image_path = self.image_path[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # needed for gradcam.
        original_image = cv2.resize(
            image, (TRANSFORMS.image_size, TRANSFORMS.image_size)
        ).copy()

        # Get target for all modes except for test, if test, replace target with dummy ones to pass through return_dtype.
        target = self.targets[index] if self.mode != "test" else torch.ones(1)

        if self.transforms:
            image = self.transforms(image=image)["image"]

        # TODO: Consider not returning original image if we don't need it, may cause more memory usage and speed issues?
        X, y, original_image = self.return_dtype(image, target, original_image)
        
        if self.mode in ["train", "valid"]:
            
            return {"X": X, "y": y}

        if self.mode == "test":
            return {"X": X}

        if self.mode == "gradcam":

            return {
                "X": X,
                "y": y,
                "original_image": original_image,
                "image_id": self.image_ids[index],
            }

<a href="#top">Back to top</a>

<a id="Augmentations"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Augmentations</h1>

Notice that in `get_inference_transforms` I defined many TTAs, in which all of them were copied verbatim from `get_train_transforms`, except for `VerticalFlip`. Since I put very low probability for `VerticalFlip`, I reckon I will just not include it in TTA.

One other note is that in order to make TTA results deterministic, I will use p=1. for all TTAs. However, if you have `RandomResizedCrops`, then there will be some randomness.

Here is a snippet of what the function looks like:

```python
def get_inference_transforms(image_size: int = TRANSFORMS.image_size) -> Dict[str, albumentations.Compose]:
    """Performs Augmentation on test dataset.
    Returns the transforms for inference in a dictionary which can hold TTA transforms.

    Args:
        image_size (int, optional): [description]. Defaults to AUG.image_size.

    Returns:
        Dict[str, albumentations.Compose]: [description]
    """

    transforms_dict = {
        "transforms_test": albumentations.Compose(
            [
                albumentations.Resize(image_size, image_size),
                albumentations.Normalize(
                    mean=TRANSFORMS.mean,
                    std=TRANSFORMS.std,
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(p=1.0),
            ]
        ),
        "tta_flip": albumentations.Compose(
            [
                albumentations.HorizontalFlip(p=1),
                albumentations.Resize(image_size, image_size),
                albumentations.Normalize(
                    mean=TRANSFORMS.mean,
                    std=TRANSFORMS.std,
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(p=1.0),
            ]
        ),
        "tta_rotate": albumentations.Compose(
            [
                albumentations.Rotate(limit=180, p=1),
                albumentations.Resize(image_size, image_size),
                albumentations.Normalize(
                    mean=TRANSFORMS.mean,
                    std=TRANSFORMS.std,
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(p=1.0),
            ]
        )
    }

    return transforms_dict
```

In [ ]:
from albumentations.pytorch.transforms import ToTensorV2
import albumentations


def get_train_transforms(image_size: int = TRANSFORMS.image_size):
    """Performs Augmentation on training data.

    Args:
        image_size (int, optional): [description]. Defaults to AUG.image_size.

    Returns:
        [type]: [description]
    """
    return albumentations.Compose(
        [  # albumentations.RandomResizedCrop(height=image_size, width=image_size),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.1),
            albumentations.Rotate(limit=180, p=0.5),
            albumentations.ShiftScaleRotate(
                shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.5
            ),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2,
                sat_shift_limit=0.2,
                val_shift_limit=0.2,
                p=0.5,
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
            ),
            albumentations.Resize(image_size, image_size),
            albumentations.Normalize(
                mean=TRANSFORMS.mean,
                std=TRANSFORMS.std,
                max_pixel_value=255.0,
                p=1.0,
            ),
            ToTensorV2(p=1.0),
        ]
    )


def get_valid_transforms(image_size: int = TRANSFORMS.image_size):
    """Performs Augmentation on validation data.

    Args:
        image_size (int, optional): [description]. Defaults to AUG.image_size.

    Returns:
        [type]: [description]
    """
    return albumentations.Compose(
        [
            albumentations.Resize(image_size, image_size),
            albumentations.Normalize(
                mean=TRANSFORMS.mean,
                std=TRANSFORMS.std,
                max_pixel_value=255.0,
                p=1.0,
            ),
            ToTensorV2(p=1.0),
        ]
    )


def get_gradcam_transforms(image_size: int = TRANSFORMS.image_size):
    """Performs Augmentation on gradcam data.

    Args:
        image_size (int, optional): [description]. Defaults to AUG.image_size.

    Returns:
        [type]: [description]
    """
    return albumentations.Compose(
        [
            albumentations.Resize(image_size, image_size),
            albumentations.Normalize(
                mean=TRANSFORMS.mean,
                std=TRANSFORMS.std,
                max_pixel_value=255.0,
                p=1.0,
            ),
            ToTensorV2(p=1.0),
        ]
    )


def get_inference_transforms(image_size: int = TRANSFORMS.image_size) -> Dict[str, albumentations.Compose]:
    """Performs Augmentation on test dataset.
    Returns the transforms for inference in a dictionary which can hold TTA transforms.

    Args:
        image_size (int, optional): [description]. Defaults to AUG.image_size.

    Returns:
        Dict[str, albumentations.Compose]: [description]
    """

    transforms_dict = {
        "transforms_test": albumentations.Compose(
            [
                albumentations.Resize(image_size, image_size),
                albumentations.Normalize(
                    mean=TRANSFORMS.mean,
                    std=TRANSFORMS.std,
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(p=1.0),
            ]
        ),
        "tta_hflip": albumentations.Compose(
            [
                albumentations.HorizontalFlip(p=1),
                albumentations.Resize(image_size, image_size),
                albumentations.Normalize(
                    mean=TRANSFORMS.mean,
                    std=TRANSFORMS.std,
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(p=1.0),
            ]
        ),
            
        "tta_rotate": albumentations.Compose(
            [
                albumentations.Rotate(limit=180, p=1),
                albumentations.Resize(image_size, image_size),
                albumentations.Normalize(
                    mean=TRANSFORMS.mean,
                    std=TRANSFORMS.std,
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(p=1.0),
            ]
        ),
        "tta_shift_scale_rotate": albumentations.Compose(
            [
                albumentations.ShiftScaleRotate(
                    shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=1
                ),
                albumentations.Resize(image_size, image_size),
                albumentations.Normalize(
                    mean=TRANSFORMS.mean,
                    std=TRANSFORMS.std,
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(p=1.0),
            ]
        ),
        "tta_hue_saturation_value": albumentations.Compose(
            [
                albumentations.HueSaturationValue(
                    hue_shift_limit=0.2,
                    sat_shift_limit=0.2,
                    val_shift_limit=0.2,
                    p=1,
                ),
                albumentations.Resize(image_size, image_size),
                albumentations.Normalize(
                    mean=TRANSFORMS.mean,
                    std=TRANSFORMS.std,
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(p=1.0),
            ]
        ),
        "tta_random_brightness_contrast": albumentations.Compose(
            [
                albumentations.RandomBrightnessContrast(
                    brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=1
                ),
                albumentations.Resize(image_size, image_size),
                albumentations.Normalize(
                    mean=TRANSFORMS.mean,
                    std=TRANSFORMS.std,
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(p=1.0),
            ]
        ),
    }

    return transforms_dict


<a href="#top">Back to top</a>

<a id="Model"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Model Instantiation</h1>

We are using custom activation in our custom `head`. A `head` is a jargon for the `classifier` layer in a CNN. In general, people finetune a pretrained model by simply removing the last classifier layer and replace it with the correct number of classes.

---

Note carefully here when you instantiate your model during **inference**. You need to do a few things:

1. Copy the exact same model over! It might seem obvious, but when I was a beginner, I was only forking other people's work, and do not know you need the exact same model to inference!
2. Set `pretrained=False`, technically, you do not need to since you are loading model from a checkpoint, but remember about Kaggle competition being mostly offline? If you set `pretrained=True`, then you need internet access.

In [ ]:
sigmoid = torch.nn.Sigmoid()

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))


class Swish_Module(torch.nn.Module):
    def forward(self, x):
        return Swish.apply(x)

Unfortunately, defining a `model` class may not necessarily be enough for reusability in every competition. However, a generic architecture as presented below should suffice for most.

In [ ]:
class CustomNeuralNet(torch.nn.Module):
    def __init__(
        self,
        model_name: str = MODEL_PARAMS.model_name,
        out_features: int = MODEL_PARAMS.output_dimension,
        in_channels: int = MODEL_PARAMS.input_channels,
        pretrained: bool = MODEL_PARAMS.pretrained,
    ):
        """Construct a new model.
        Args:
            model_name ([type], str): The name of the model to use. Defaults to MODEL_PARAMS.model_name.
            out_features ([type], int): The number of output features, this is usually the number of classes, but if you use sigmoid, then the output is 1. Defaults to MODEL_PARAMS.output_dimension.
            in_channels ([type], int): The number of input channels; RGB = 3, Grayscale = 1. Defaults to MODEL_PARAMS.input_channels.
            pretrained ([type], bool): If True, use pretrained model. Defaults to MODEL_PARAMS.pretrained.
        """
        super().__init__()

        self.in_channels = in_channels
        self.pretrained = pretrained

        self.backbone = timm.create_model(
            model_name, pretrained=self.pretrained, in_chans=self.in_channels
        )
        config.logger.info(
            f"\nModel: {model_name} \nPretrained: {pretrained} \nIn Channels: {in_channels}"
        )

        # removes head from backbone
        self.backbone.reset_classifier(num_classes=0, global_pool="avg")

        # get the last layer's number of features in backbone (feature map)
        self.in_features = self.backbone.num_features
        self.out_features = out_features

        # Custom Head
        self.single_head_fc = torch.nn.Sequential(
            torch.nn.Linear(self.in_features, self.out_features),
        )

        self.architecture: Dict[str, Callable] = {
            "backbone": self.backbone,
            "bottleneck": None,
            "head": self.single_head_fc,
        }

    def extract_features(self, image: torch.FloatTensor) -> torch.FloatTensor:
        """Extract the features mapping logits from the model.
        This is the output from the backbone of a CNN.
        Args:
            image (torch.FloatTensor): The input image.
        Returns:
            feature_logits (torch.FloatTensor): The features logits.
        """
        feature_logits = self.architecture["backbone"](image)
        return feature_logits

    def forward(self, image: torch.FloatTensor) -> torch.FloatTensor:
        """The forward call of the model.
        Args:
            image (torch.FloatTensor): The input image.
        Returns:
            classifier_logits (torch.FloatTensor): The output logits of the classifier head.
        """

        feature_logits = self.extract_features(image)
        classifier_logits = self.architecture["head"](feature_logits)

        return classifier_logits

    def get_last_layer(self):
        # TODO: Implement this properly.
        """Get the last layer information of TIMM Model.
        Returns:
            [type]: [description]
        """
        last_layer_name = None
        for name, _param in self.model.named_modules():
            last_layer_name = name

        last_layer_attributes = last_layer_name.split(".")  # + ['in_features']
        linear_layer = functools.reduce(
            getattr, last_layer_attributes, self.model
        )
        # reduce applies to a list recursively and reduce
        in_features = functools.reduce(
            getattr, last_layer_attributes, self.model
        ).in_features
        return last_layer_attributes, in_features, linear_layer


<a href="#top">Back to top</a>

<a id="OOF"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Out-of-Fold</h1>

If you did not save OOF predictions during training, then the only way to recover back is to predict on each fold again! 

<a id="8"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Inference By Folds</h1>

Inference by fold function has remained faithful to me throughout competitions, I only ever have to worry about changing `sigmoid` to `softmax` depending on the model's settings. One thing to note is that, I save a lot of things in the `model` `state_dict`. In particular, I save `oof` predictions in my `state_dict`. This makes me pulling out `oof` predictions easy when I accidentally didn't manage to save my `oof` during training, which happens often.

Go down to my example section for better understanding how it works under the hood.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import glob

def inference_all_folds(
    model: CustomNeuralNet,
    state_dicts: List[collections.OrderedDict],
    test_loader: torch.utils.data.DataLoader,
) -> np.ndarray:
    """Inference the model on all K folds.

    Args:
        model (models.CustomNeuralNet): The model to be used for inference. Note that pretrained should be set to False.
        state_dicts (List[collections.OrderedDict]): The state dicts of the models. Generally, K Fold means K state dicts.
        test_loader (torch.utils.data.DataLoader): The dataloader for the test set.

    Returns:
        mean_preds (np.ndarray): The mean of the predictions of all folds.
    """

    model.to(device)
    model.eval()

    with torch.no_grad():
        all_folds_preds = []

        for _fold_num, state in enumerate(state_dicts):
            if "model_state_dict" not in state:
                model.load_state_dict(state)
            else:
                model.load_state_dict(state["model_state_dict"])

            current_fold_preds = []

            for data in tqdm(test_loader, position=0, leave=True):
                images = data["X"].to(device, non_blocking=True)
                logits = model(images)
                test_prob = logits.sigmoid().cpu().numpy()
                test_prob = test_prob * 100 # Only for petfinder

                current_fold_preds.append(test_prob)

            current_fold_preds = np.concatenate(current_fold_preds, axis=0)
            all_folds_preds.append(current_fold_preds)
        mean_preds = np.mean(all_folds_preds, axis=0)
    return mean_preds


def inference(
    df_test: pd.DataFrame,
    model_dir: str,
    model: CustomNeuralNet = CustomNeuralNet(pretrained=False),
    df_sub: pd.DataFrame = None,
    transform_dict: Dict[str, albumentations.Compose] = get_inference_transforms()
) -> Dict[str, np.ndarray]:
    """Inference the model and perform TTA, if any.

    Dataset and Dataloader are constructed within this function because of TTA.

    Args:
        df_test (pd.DataFrame): The test dataframe.
        model_dir (str): model directory for the model.
        df_sub (pd.DataFrame, optional): The submission dataframe. Defaults to None.

    Returns:
        all_preds (Dict[str, np.ndarray]): {"normal": normal_preds, "tta": tta_preds}
    """

    if df_sub is None:
        config.logger.info(
            "No submission dataframe detected, setting df_sub to be df_test."
        )
        df_sub = df_test.copy()

    all_preds = {}

    # model = CustomNeuralNet(pretrained=False).to(device)

    # transform_dict = get_inference_transforms()

    weights = sorted(list([model_path for model_path in glob.glob(model_dir + "/*.pt")]))
    
#     weights = ["../input/swin-transformer-mixup/model/swin_large_patch4_window7_224/swin_large_patch4_window7_224_best_valid_rmse_fold_1.pt",
#                 "../input/swin-transformer-mixup/model/swin_large_patch4_window7_224/swin_large_patch4_window7_224_best_valid_rmse_fold_2.pt",
#                "../input/swin-transformer-mixup/model/swin_large_patch4_window7_224/swin_large_patch4_window7_224_best_valid_rmse_fold_3.pt",
#                "../input/swin-transformer-mixup/model/swin_large_patch4_window7_224/swin_large_patch4_window7_224_best_valid_rmse_fold_4.pt",
#                "../input/swin-transformer-mixup/model/swin_large_patch4_window7_224/swin_large_patch4_window7_224_best_valid_rmse_fold_5.pt"
#               ]


    state_dicts = [torch.load(path)["model_state_dict"] for path in weights]
    preds_list = []
    # Loop over each TTA transforms, if TTA is none, then loop once over normal inference_augs.
    for aug_name, aug_param in transform_dict.items():
        test_dataset = CustomDataset(
            df=df_test, transforms=aug_param, mode="test"
        )
        test_loader = torch.utils.data.DataLoader(
            test_dataset, **LOADER_PARAMS.test_loader
        )
        predictions = inference_all_folds(
            model=model, state_dicts=state_dicts, test_loader=test_loader
        )

        all_preds[aug_name] = predictions

        ################# To change when necessary depending on the metrics needed for submission #################
        df_sub[FOLDS.class_col_name] = predictions

        df_sub[[FOLDS.image_col_name, FOLDS.class_col_name]].to_csv(
            f"submission_{aug_name}.csv", index=False
        )
        preds_list.append(predictions)
        print(df_sub.head())

        plt.figure(figsize=(12, 6))
        plt.hist(df_sub[FOLDS.class_col_name], bins=100)
    # For petfinder
    df_sub[FOLDS.class_col_name] = np.mean(np.stack(preds_list).squeeze(axis=2), axis=0)
    df_sub[[FOLDS.image_col_name, FOLDS.class_col_name]].to_csv(
            f"submission.csv", index=False
        )
    return all_preds, preds_list

In [ ]:
is_inference = False

if is_inference:
    model_dir = "../input/swin-transformer-mixup/model/swin_large_patch4_window7_224"
    predictions, _ = inference(df_test, model_dir, df_sub)

<a href="#top">Back to top</a>

<a id="PP"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Ensembling Techniques</h1>

## Forward Ensembling

Will add in soon!

For now, we do a simple weighted mean ensemble!

In practice, if we have 2 models, then we should have 2 callable config file, say a `yaml` file, and we can call them separately. In notebook, we just do the following.

In [ ]:
tf_efficientnet_b4_ns_1llvj0qo = CustomNeuralNet(
    model_name="tf_efficientnet_b4_ns",
    out_features=1,
    in_channels=3,
    pretrained=False,
)

tf_efficientnet_b4_ns_1llvj0qo_dir = "../input/petfinder-tf-efficientnet-b4-ns-5-folds-1llvj0qo/model/tf_efficientnet_b4_ns"
tf_efficientnet_b4_ns_1llvj0qo_transform_dict = get_inference_transforms(
    image_size=448
)

_, tf_efficientnet_b4_ns_1llvj0qo_preds_list = inference(
    df_test,
    tf_efficientnet_b4_ns_1llvj0qo_dir,
    tf_efficientnet_b4_ns_1llvj0qo,
    df_sub,
    tf_efficientnet_b4_ns_1llvj0qo_transform_dict
)


In [ ]:
swin_large_patch4_window7_224_246zq00g = CustomNeuralNet(
    model_name="swin_large_patch4_window7_224",
    out_features=1,
    in_channels=3,
    pretrained=False,
)
swin_large_patch4_window7_224_246zq00g_dir = (
    "../input/swin-transformer-mixup/model/swin_large_patch4_window7_224"
)

swin_large_patch4_window7_224_246zq00g_transform_dict = (
    get_inference_transforms(image_size=224)
)


_, swin_large_patch4_window7_224_246zq00g_preds_list = inference(
    df_test,
    swin_large_patch4_window7_224_246zq00g_dir,
    swin_large_patch4_window7_224_246zq00g,
    df_sub,
    swin_large_patch4_window7_224_246zq00g_transform_dict
)


In [ ]:
df_sub[FOLDS.class_col_name] = np.mean(np.stack(tf_efficientnet_b4_ns_1llvj0qo_preds_list).squeeze(axis=2), axis=0) * 0.33 + np.mean(np.stack(swin_large_patch4_window7_224_246zq00g_preds_list).squeeze(axis=2), axis=0) * 0.67
df_sub[[FOLDS.image_col_name, FOLDS.class_col_name]].to_csv(
        f"submission.csv", index=False
    )

<a id="PP"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Post-Processing Techniques</h1>

## Training Image Embeddings

### CatBoost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
num_image_embeddings = 1536 # hardcoded the image embeds for swin transformer
num_image_embeddings = 1792 # tf_efficientnet_b4_ns

In [ ]:
# weights = ["../input/swin-transformer-mixup/model/swin_large_patch4_window7_224/swin_large_patch4_window7_224_best_valid_rmse_fold_1.pt",
#             "../input/swin-transformer-mixup/model/swin_large_patch4_window7_224/swin_large_patch4_window7_224_best_valid_rmse_fold_2.pt",
#            "../input/swin-transformer-mixup/model/swin_large_patch4_window7_224/swin_large_patch4_window7_224_best_valid_rmse_fold_3.pt",
#            "../input/swin-transformer-mixup/model/swin_large_patch4_window7_224/swin_large_patch4_window7_224_best_valid_rmse_fold_4.pt",
#            "../input/swin-transformer-mixup/model/swin_large_patch4_window7_224/swin_large_patch4_window7_224_best_valid_rmse_fold_5.pt"
#           ]

weights = ["../input/petfinder-tf-efficientnet-b4-ns-5-folds-1llvj0qo/model/tf_efficientnet_b4_ns/tf_efficientnet_b4_ns_best_valid_rmse_fold_1.pt",
          "../input/petfinder-tf-efficientnet-b4-ns-5-folds-1llvj0qo/model/tf_efficientnet_b4_ns/tf_efficientnet_b4_ns_best_valid_rmse_fold_2.pt",
          "../input/petfinder-tf-efficientnet-b4-ns-5-folds-1llvj0qo/model/tf_efficientnet_b4_ns/tf_efficientnet_b4_ns_best_valid_rmse_fold_3.pt",
          "../input/petfinder-tf-efficientnet-b4-ns-5-folds-1llvj0qo/model/tf_efficientnet_b4_ns/tf_efficientnet_b4_ns_best_valid_rmse_fold_4.pt",
          "../input/petfinder-tf-efficientnet-b4-ns-5-folds-1llvj0qo/model/tf_efficientnet_b4_ns/tf_efficientnet_b4_ns_best_valid_rmse_fold_5.pt",]
batch_size = 8

In [ ]:
inference_embeddings = False

if inference_embeddings:

    # Utimately we want embeddings to be shape (n_samples, n_embeddings)
    # SO in my case if n_embeddings=1536, then if 100 samples, our final shape is (100, 1536)
    for index, fold in enumerate(range(1, FOLDS.num_folds+1)):
        print('#'*25)
        print('### FOLD',fold)
        print('#'*25)


        df_valid = df_folds[df_folds["fold"] == fold].reset_index(drop=True)
        # Here hard code the embeddings first, as ultimately we want (n_samples, 1536)
        embeddings = torch.zeros(df_valid.shape[0], num_image_embeddings)

        model = CustomNeuralNet(pretrained=False).to(device)

        valid_transforms = get_inference_transforms()["transforms_test"]

        state_dict = torch.load(weights[index])["model_state_dict"]
        model.load_state_dict(state_dict)
        model.eval() # Think if u extract embeddings should set to eval too

        valid_dataset = CustomDataset(
            df=df_valid, transforms=valid_transforms, mode="train"
        )
        valid_loader = torch.utils.data.DataLoader(
            valid_dataset, shuffle=False, batch_size = batch_size
        )
        row, col = 0, batch_size
        with torch.no_grad():
            for data in tqdm(valid_loader, position=0, leave=True):
                images = data["X"].to(device)
                targets = data["y"].to(device)
                # SWIN returns 1536 as embeddings, depends on how you take it it can be flattened or not yet
                curr_batch_embeds = model.extract_features(images)
                embeddings[row:col] = curr_batch_embeds
                row, col = row + batch_size, col + batch_size

        with open(f"fold_{fold}_image_embeddings.npy", 'wb') as f:
            np.save(f, embeddings.cpu().detach().numpy().astype('float32'))

    image_embeddings_paths = [
        "fold_1_image_embeddings.npy",
        "fold_2_image_embeddings.npy",
        "fold_3_image_embeddings.npy",
        "fold_4_image_embeddings.npy",
        "fold_5_image_embeddings.npy",
    ]

    image_embeddings_per_fold = [np.load(image_embeddings_path) for image_embeddings_path in image_embeddings_paths]

    print(image_embeddings_per_fold[0].shape)

    concat_all_image_embeddings = np.concatenate(image_embeddings_per_fold, axis=0) # fold 1-5 all 

    # One important thing is you need the `df_folds` to be the same as you trained them in order to recover the image embeddings and its corresponding labels.
    # we want our oof np arrays to be of shape (num_folds, num_samples) where 1st row is the preds of the first fold etc 
    # oof_trues, oof_preds = np.zeros(shape=(FOLDS.num_folds, df_folds.shape[0])), np.zeros(shape=(FOLDS.num_folds, df_folds.shape[0]))
    # print(oof_trues.shape)

    final_oof_preds, final_oof_trues = [], []

    df_folds['stratify_label'] = pd.qcut(df_folds['Pawpularity'], q = 30, labels = range(30))


    # skf = StratifiedKFold(n_splits = 6, shuffle = True, random_state = FOLDS.seed)
    # Y_strat = df_folds["stratify_label"].values
    # Y_pawpularity = df_folds['Pawpularity'].values * 100

    # for idx, (train, val) in enumerate(skf.split(concat_all_image_embeddings, Y_strat)):
    #         train_x, train_y = concat_all_image_embeddings[train], Y_pawpularity[train]
    #         val_x, val_y = concat_all_image_embeddings[val], Y_pawpularity[val]
    #         # Set CatBoost Parameters
    #         cb_params = {'loss_function' : 'RMSE',
    #                      'eval_metric' : 'RMSE',
    #                      'iterations' : 1000,
    #                      'grow_policy' : 'SymmetricTree',
    #                      'depth' : 6,
    #                      'l2_leaf_reg' : 2.0,
    #                      'random_strength' : 1.0,
    #                      'learning_rate' : 0.05,
    #                      'task_type' : 'CPU',
    #                      'devices' : '0',
    #                      'verbose' : 0,
    #                      'random_state': FOLDS.seed}

    #         # Create and Fit CatBoost Model
    #         cb_model = CatBoostRegressor(**cb_params)
    #         cb_model.fit(train_x, train_y, eval_set = [(val_x, val_y)], early_stopping_rounds = 100, verbose = 250)


    import optuna
    from sklearn import metrics

    def objective(trial, train_x, train_y, valid_x, valid_y):

        param = {
            # "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
            #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.05),
            "depth": trial.suggest_int("depth", 1, 12),
            "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
            "l2_leaf_reg": trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5),
            'min_child_samples' : trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32]),

    #         "bootstrap_type": trial.suggest_categorical(
    #             "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
    #         ),
            "grow_policy": "SymmetricTree",
            # "used_ram_limit": "3gb",
            "task_type": "CPU",
            "devices": "0",
            "verbose": 0,
            "random_state": FOLDS.seed
        }

    #     if param["bootstrap_type"] == "Bayesian":
    #         param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    #     elif param["bootstrap_type"] == "Bernoulli":
    #         param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

        cb_model = CatBoostRegressor(**param)

        cb_model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=100, early_stopping_rounds=100)

        valid_preds = cb_model.predict(valid_x)

        rmse = metrics.mean_squared_error(valid_y.flatten(), valid_preds.flatten(), squared=False)
        return rmse



    # Utimately we want embeddings to be shape (n_samples, n_embeddings)
    # SO in my case if n_embeddings=1536, then if 100 samples, our final shape is (100, 1536)
    for index, fold in enumerate(range(1, FOLDS.num_folds+1)):
        print('#'*25)
        print('### FOLD',fold)
        print('#'*25)


        df_train = df_folds[df_folds["fold"] != fold].reset_index(drop=True)
        df_valid = df_folds[df_folds["fold"] == fold].reset_index(drop=True)
        oof_trues, oof_preds = np.zeros(shape=(df_valid.shape[0])), np.zeros(shape=(df_folds.shape[0]))
        train_x, train_y = np.concatenate(np.delete(image_embeddings_per_fold, index, axis=0), axis=0), df_train["Pawpularity"].values * 100
        #train_x, train_y = image_embeddings_per_fold[2], df_folds[df_folds["fold"] == 3].reset_index(drop=True)["Pawpularity"].values * 100

        valid_x, valid_y = image_embeddings_per_fold[index], df_valid["Pawpularity"].values * 100
        study = optuna.create_study(study_name=f'catboost-seed{FOLDS.seed}')
        # https://optuna.readthedocs.io/en/stable/faq.html
        study.optimize(lambda trial: objective(trial, train_x, train_y, valid_x, valid_y), n_trials=10000, n_jobs=-1, timeout=24000)
        break
    #     # Set CatBoost Parameters
    #     cb_params = {'loss_function' : 'RMSE',
    #                  'eval_metric' : 'RMSE',
    #                  'iterations' : 1000,
    #                  'grow_policy' : 'SymmetricTree',
    #                  'depth' : 6,
    #                  'l2_leaf_reg' : 2.0,
    #                  'random_strength' : 1.0,
    #                  'learning_rate' : 0.05,
    #                  'task_type' : 'CPU',
    #                  'devices' : '0',
    #                  'verbose' : 0,
    #                  'random_state': FOLDS.seed}

    #     # Create and Fit CatBoost Model
    #     cb_model = CatBoostRegressor(**cb_params)
    #     cb_model.fit(train_x, train_y, eval_set = [(valid_x, valid_y)], early_stopping_rounds = 1000, verbose = 250)

    #     valid_preds = cb_model.predict(valid_x)
    #     oof_trues[index] = valid_y # just assign it to the index, simple, oof_trues[0] -> first fold etc 
    #     oof_preds[index] = valid_preds


    #     pickle.dump(cb_model, open(f"catboost_fold_{fold}", "wb"))


    #     final_oof_trues.append(oof_trues)
    #     final_oof_preds.append(oof_preds)
    #     # Cleanup
    #     del cb_model, valid_preds
    #     del train_x, train_y
    #     del valid_x, valid_y
    #     gc.collect()  

    final_oof_trues = final_oof_trues.flatten()
    final_oof_preds = final_oof_preds.flatten()


    metrics.mean_squared_error(final_oof_trues, final_oof_preds, squared=False)

<a id="60"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Dissecting Inference By Folds</h1>

In Kaggle, the idea of K-Fold cross validation is rampant, and rightfully so. We take an extract from @cpmp below with original [discussion here](https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/275883#1533680):

> K-Folds cross validation was devised as a way to assess model performance using training data. A great paper on this from Sebastian Raschka is a must read https://arxiv.org/abs/1811.12808. You use k folds cv to tune you model, then retrain on all training data with best found tuning. However, once you have run K-Fold CV, you get K trained models. Kagglers quickly found that ensembling these models was giving good results at zero computation cost, rather than having to retrain a model on full data. It soon became a very common practice.

Thus, as a newbie last time, I was not at all familiar with the idea of inferencing five folds. I was more accustomed to the fact of say, training and **validation** with K-Folds to **obtain best hyperparameters** of a model, where the process is often called Model Selection, and then we use the best found hyperparameters $\mathcal{C}$ to perform a full training on the whole training dataset (train+validation) $\mathcal{X}$, and subsequently we use the parameters of the re-trained model as the final set of weights and inference on subsequent test set. **Note: hyperparameters and parameters are different! Do not get confused!**

Below we will break down what `inference_all_folds` does!

<a id="61"></a>

<h2 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Problem Settings</h2>

### Classification Problem with Softmax Activation

The below problem illustrates a typical multiclass classification problem, which uses a `softmax` activation. The problem will mostly be the same if we use `sigmoid` activation as well.

---

- Batch Size: 2
- Number of Test Images: 3 (img1, img2, img3)
- Number of Folds/Weights: 5 (f1, f2, f3, f4, f5)
- Number of classes: 5 (c1, c2, c3, c4, c5)

---



Let us walk through the typical inference pipeline in PyTorch step-by-step!

<a href="#top">Back to top</a>

###  Initial steps for Inference

1. We call the model first:

```python
model = CustomNeuralNet(pretrained=False).to(device)
```

Note that the emphasis is you need to use back the exact same model that you used for training during inference, and, for most cases, internet is off, so switch `pretrained` to `False`. Lastly, the usual drill for PyTorch, we need to put our model on to `device`, a flag to indicate whether our system is in GPU or CPU mode.

---

2. We then define the `datasets` as such:

```python
test_dataset = CustomDataset(df=df_test, transforms=aug_param, mode="test")
test_loader = torch.utils.data.DataLoader(test_dataset, **LOADER_PARAMS.test_loader)
```

Here we need to understand a few things:
    - `df_test`: Your test dataframe.
    - `mode`: Set to `test` during inference, this is on how you create your `dataset`, but in general, you can follow my template.
    - `transforms=aug_param`: As of now, just need to know that we should pass in the inference transforms, something very important here is, that you need to pass in the exact same `Normalization` parameters you used for training! In addition, it is also recommended to inference at the same image size you trained on; some of us may be adventurous, and use a bigger image size during inference. We will not go into TTA now, so hold your horses.
```python
albumentations.Resize(image_size, image_size),
albumentations.Normalize(same as training)
```

Last but not least, equip your `dataset` with `DataLoader` and you are good to go! Note that the fancy looking notation `**LOADER_PARAMS.test_loader**` is just me loading off some pre-defined parameters for the DataLoader. If in doubt, print out `LOADER_PARAMS.test_loader` and you will know what I am saying.

---

3. Let us see what's next, we need to define and load up the weights from our trained models! Since we trained five folds, then there should be 5 sets of weights.

```python
weights = [model_path for model_path in glob.glob(model_dir + "/*.pt")] = [weight_for_fold_1, weight_for_fold_2, ..., weight_for_fold_5]
```

The `weights` variable merely holds the `path` of the saved weights, we now need to **load** them as PyTorch's state dicts, where you can read up more below.

```
state_dicts = [torch.load(path)["model_state_dict"] for path in weights]
```

Now, one may wonder why there is `model_state_dict` after loading the path, this is unique to each individual, when you save a model in PyTorch, you can assign a key (name) to the ordered dict, and to identify it easily, I called it `model_state_dict`. You will also read more about it in details below.

---

### Inference All Folds

4. We are ready to move on to my main function, named `inference_all_folds`.
```python
def inference_all_folds(
    model: CustomNeuralNet,
    state_dicts: List[collections.OrderedDict],
    test_loader: torch.utils.data.DataLoader,
) -> np.ndarray:
```
As we see, my function takes in three arguments, all of which were defined accordingly from Step 1 to 3. So let's go!

---

5. We set `model` to `eval()` mode as we are in inference phase; then since we are inference phase, we say `torch.no_grad()` because we are no longer computing or storing gradients anymore. Usually the three buddies are as follows:

```python
model.to(device)
model.eval()
with torch.no_grad():
    inference code...
```

### What happens in the loop?

We will dissect step by step on what happens in the inference loop, firstly, we have **3 images and our batch size is 2**, this means that our `DataLoader` needs 2 loops to finish inference. Now, this was designed on purpose to see how the loop works, if not, I can easily set the batch size to be 4, or 3 for that matter to let our inference be done in 1 single loop.

#### First loop

We list what our first loop consists of, we will indicate the explaination below in line number. Note that the first loop is loop over `state_dicts`, in other words, the first loop is by folds, first loop -> first fold, and so on.

```python
with torch.no_grad():
    all_folds_preds = []
    for _fold_num, state in enumerate(state_dicts):
        if "model_state_dict" not in state:
            model.load_state_dict(state)
        else:
            model.load_state_dict(state["model_state_dict"])
        current_fold_preds = []
```

6. **Line 2:** Initiate with an empty list `all_folds_preds=[]`, where the final expectation of this `list` should contain 5 `numpy array` with each `array` having a shape of `3 by 5`, or `n by 5` where `n` represents the total number of test images. 

7. **Line 3-7:** Here is where we start looping through each fold's model's and make predictions; since there are 5 folds, this implies we have 5 models, and as mentioned earlier, `states_dicts` is a `list` holding 5 `ordered dict (information of each model's weights)` of each model. The subsequent `if-else` clause is to tell the model to load our state dict, intuitively, it means now our initiated model's weights (pretrained = False) are now replaced with the weights trained by us.

8. **Line 8:** Initiate with another empty list: `current_fold_preds = []` where the end result of it is all the predictions of fold $i$ (F1,2,3,4,5) contained in this list in the following format:

```python
current_fold_preds = [
    [[predictions_for_img1_for_fold_i], [predictions_for_img2_for_fold_i]],
    [predictions_for_img3_for_fold_i],
]
```
Note that the shape of `[predictions_for_img1_for_fold_i]` may change depending on your activation function, and the number of classes, here we use softmax and has 5 classes. Something very interesting is that the `current_fold_preds` is a list of 2 differently shaped numpy arrays, the first entry is of shape `(2,5)`, and the second entry is an array of shape `(1,5)`. When stacked/concat together, we should have a numpy array of shape `(3,5)`. This is not surprising, since we have 3 test images, and hence 3 predictions, which is the rows, and 5 columns for each row is because of we have 5 classes, and each column is the "probability/predictions" for class $c_i$. Furthermore, the reason of them being split up is because we set the `batch_size` to be 2, so when you iterate through the `DataLoader`, the first for loop will append the first 2 images' predictions, and the second loop will append the remaining 1 image's predictions.
    
#### Second Loop

The second loop, is looping over our `DataLoader`. Since we have 3 images and batch size of 2, we will loop over the loader twice in each fold.  In essence, we loop through the `DataLoader/TestLoader` and predict for each batch of images by calling `model(images)`, which will in turn return you **logits** because that is how we defined it in our **Model**. Subsequently, we convert the **logits** into **softmax predictions**. The naming might be not suggestive enough, but both **logits and softmax_preds** are **tensor array and numpy array respectively**.

```python
for data in tqdm(test_loader, position=0, leave=True):
    images = data["X"].to(device, non_blocking=True)
    logits = model(images)
    test_prob = torch.nn.Softmax(dim=1)(input=logits).to('cpu').numpy() # logits.sigmoid().cpu().numpy()

    current_fold_preds.append(test_prob)
```
    
9. **Line 2-3:** We get our images and call our model on the set of images, just like how we did in our training pipeline.

10. **Line 4:** This is just making predictions for each batch of images, note one needs to use the exact same activation as you did in training. Usually, in classification, its either `softmax` or `sigmoid`.

11. **Line 5:** We append the predictions/probs to `current_fold_preds`, one can now connect this back with point 8.


#### End of (Outer) Loop

```python
current_fold_preds = np.concatenate(current_fold_preds, axis=0)
all_folds_preds.append(current_fold_preds)
```

12. After finishing each inner loop, we then `concatenate` the `current_fold_preds` and `append` it to `all_folds_preds`. The reason for `concatenate` is to convert the `list` into `array` as follows:

```python
# After fold 1's inference, we have
current_fold_preds = [
    [[0.01, 0.02, 0.03, 0.9, 0.04], [0.02, 0.8, 0.05, 0.06, 0.07]],
    [0.01, 0.8, 0.03, 0.09, 0.07],
]

current_fold_preds = np.concatenate(current_fold_preds, axis=0)
current_fold_preds = [
    [0.01, 0.02, 0.03, 0.9, 0.04],
    [0.02, 0.8, 0.05, 0.06, 0.07],
    [0.01, 0.8, 0.03, 0.09, 0.07],
]
all_folds_preds.append(current_fold_preds)
all_folds_preds = [current_fold_preds (fold_1_preds)]
```

Therefore, after five folds, we have something like this:

```python
all_folds_preds = [fold_1_preds, fold_2_preds, ..., fold_5_preds] -> a list of 5 numpy arrays of shape 3x5
```

#### End of Both Loops

13. At this junction, we are edging towards the end. We will do a summary as follows, along with the last line of code: `mean_preds = np.mean(all_folds_preds, axis=0)`.
    - Outer Loop: We loop through each model's (5 folds = 5 models) `states_dicts` and for each fold/model,
    - Inner Loop: We loop through the `DataLoader` to predict all the images in the test set.
    - End of Inner Loop: For each `end`, we have `current_fold_preds` for f1, f2, ...
    - End of Outer Loop: `all_folds_preds = [fold_1_preds, fold_2_preds, fold_3_preds, fold_4_preds, fold_5_preds]`
    - Finally, we average the `all_folds_preds` using `avg_preds = np.mean(all_folds_preds, axis=0)` to get our averaged predictions, it is worth mentioning that now `all_folds_preds`, if treated as a numpy array, has shape of (5, 3, 5) where dimension/axis 0 is the fold number, dimension/axis 1 and 2 are just the `current_fold_preds`. So it is simply a numpy array of 5 sets of 3x5 predictions. We there just need to average over dimension/axis 0 to get an average over folds! As simple as that!
    ```python
    all_folds_preds = np.array(
        [
            np.array(
                [
                    [0.01, 0.02, 0.03, 0.9, 0.04],
                    [0.02, 0.8, 0.05, 0.06, 0.07],
                    [0.01, 0.8, 0.03, 0.09, 0.07],
                ]
            ),
            np.array(
                [
                    [0.03, 0.05, 0.01, 0.88, 0.03],
                    [0.01, 0.82, 0.02, 0.07, 0.08],
                    [0.005, 0.81, 0.0205, 0.0555, 0.109],
                ]
            ),
            np.array(
                [
                    [0.05, 0.03, 0.08, 0.83, 0.01],
                    [0.05, 0.89, 0.01, 0.02, 0.03],
                    [0.03, 0.78, 0.05, 0.02, 0.12],
                ]
            ),
            np.array(
                [
                    [0.02, 0.01, 0.03, 0.92, 0.02],
                    [0.05, 0.85, 0.01, 0.01, 0.08],
                    [0.02, 0.88, 0.03, 0.05, 0.02],
                ]
            ),
            np.array(
                [
                    [0.01, 0.02, 0.02, 0.93, 0.02],
                    [0.03, 0.76, 0.06, 0.06, 0.09],
                    [0.02, 0.83, 0.01, 0.07, 0.07],
                ]
            ),
        ]
    )
    # we use np.mean on axis=0 to calculate the average. axis=0 just means we take the mean of each row.
    mean_preds = np.mean(all_folds_preds, axis=0)
    ```    
    
    If you only pass in one fold/model, this inference function will still work.

- all_preds_array: each inner array is 2d and is of shape (3, 5) - 3 rows and 5 columns where 3 is the number of images, and 5 the number of predictions per image.

I purposely made the `batch_size` to be 2 and total number of images to predict on to be 3. This is to tell you that your `test_loader` will take two loops to complete the predictions.

The below `all_preds_array` is a dummy predictions I made on the 3 images across 5 folds/models. Note that there are 5 inner arrays in this, as an example, we can take `all_preds_array[0]` to be the first fold predictions across all 3 images. This inner array is a 2d array of 3 by 5, where the first row, `all_preds_array[0][0]` is the 5 predictions outputed by `softmax` for image number 1. And to be more pedantic,`all_preds_array[0][0]` being 0.01 just means for image 1, fold 1 predicts the image's probability to be class 1 is 0.01, and `all_preds_array[0][1]` just mean for the same image 1, fold 1 predicts the image's probability to be class 2 is 0.02. Note that if you sum up each row, it must add up to 1 in this case, because we are using `softmax`, where the predictions for all 5 class must sum up to 1.

---

So we have 5 folds, and let us put focus on just image 1 for simplicity. We then have perform inference 5 times on image 1 using the same model, holding everything else constant. 

- Image 1 Fold 1 Predictions: `[0.01, 0.02, 0.03, 0.9, 0.04]`
- Image 1 Fold 2 Predictions: `[0.03, 0.05, 0.01, 0.88, 0.03]`
- Image 1 Fold 3 Predictions: `[0.05, 0.03, 0.08, 0.83, 0.01]`
- Image 1 Fold 4 Predictions: `[0.02, 0.01, 0.03, 0.92, 0.02]`
- Image 1 Fold 5 Predictions: `[0.01, 0.02, 0.02, 0.93, 0.02]`

We then add all 5 folds up and **average** them. This is akin to performing a mean ensemble. In general, we average the predictions. This usually produces a more robust result (can you explain why?)

---

> So we have trained 5 models because we used 5 fold cross validation. For each of the 5 fold models that we have, we will load each fold model's weights and use them to make predictions on the unseen test images. As a result, we have 5 predictions for each test image. To be clear, if you have 1000 unseen test images, named $$i~~~ \forall i \in {1,2,...,1000}$$ then for each image $i$, there will be 5 predictions each for it, we can call them as such $$P(i_{j}) ~~~ \forall j \in {1,2,3,4,5}$$ where $j$ represents the number of folds. Thus, by convention, we take the **average/mean** of these 5 sets of predictions and take the average value/probability as the final prediction value. This score is then submitted to Kaggle and we get what we called the LB score, which should correlate to your CV/OOF score.



### Understanding with a concrete example

Unhide to see a concrete example of how each fold and predictions are being averaged.

In [ ]:
### Fold 1 ###
### First Batch: 2 Predictions ###
fold_1_pred_1 = np.array(
    [[0.01, 0.02, 0.03, 0.9, 0.04], [0.02, 0.8, 0.05, 0.06, 0.07]])

### Second Batch: 1 Prediction ###

fold_1_pred_2 = np.array([[0.01, 0.8, 0.03, 0.09, 0.07]])


### Fold 2 ###
### First Batch: 2 Predictions ###

fold_2_pred_1 = np.array([[0.03, 0.05, 0.01, 0.88, 0.03], [
                         0.01, 0.82, 0.02, 0.07, 0.08]])

### Second Batch: 1 Prediction ###

fold_2_pred_2 = np.array([[0.005, 0.81, 0.0205, 0.0555, 0.109]])


### Fold 3 ###
### First Batch: 2 Predictions ###

fold_3_pred_1 = np.array([[0.05, 0.03, 0.08, 0.83, 0.01], [
                         0.05, 0.89, 0.01, 0.02, 0.03]])

### Second Batch: 1 Prediction ###

fold_3_pred_2 = np.array([[0.03, 0.78, 0.05, 0.02, 0.12]])


### Fold 4 ###
### First Batch: 2 Predictions ###

fold_4_pred_1 = np.array([[0.02, 0.01, 0.03, 0.92, 0.02], [
                         0.05, 0.85, 0.01, 0.01, 0.08]])

### Second Batch: 1 Prediction ###

fold_4_pred_2 = np.array([[0.02, 0.88, 0.03, 0.05, 0.02]])


### Fold 5 ###
### First Batch: 2 Predictions ###

fold_5_pred_1 = np.array([[0.01, 0.02, 0.02, 0.93, 0.02], [
                         0.03, 0.76, 0.06, 0.06, 0.09]])

### Second Batch: 1 Prediction ###

fold_5_pred_2 = np.array([[0.02, 0.83, 0.01, 0.07, 0.07]])


### This list should contain the predictions of all 5 folds ###
all_folds_preds = []


fold_1_preds = []

### Fold 1 ###
### All Batches: Total of 3 Predictions in the following format ###

fold_1_preds = [fold_1_pred_1, fold_1_pred_2]
#print('\nfold_1_preds before np.concatenate\n',fold_1_preds)

### Concatenate because previous format is not good, we want it to be a numpy array ###

fold_1_preds = np.concatenate(fold_1_preds, axis=0)

# Something good to know, concatenate works exactly the same as such: #
# The idea is you concatenate two list, over the axis 0 which is rows. #

fold_1_preds_ = np.concatenate([fold_1_pred_1, fold_1_pred_2], axis=0)
#print('fold_1_preds after np.concatenate\n',fold_1_preds)
#print('fold_1_preds after np.concatenate using different method\n',fold_1_preds_)

all_folds_preds.append(fold_1_preds)
print('All Folds Pred list after Fold 1 is \n\n{}\n\n'.format(all_folds_preds))
###################################################################################

fold_2_preds = []

### Fold 2 ###
### All Batches: Total of 3 Predictions in the following format ###

fold_2_preds = [fold_2_pred_1, fold_2_pred_2]
fold_2_preds = np.concatenate(fold_2_preds, axis=0)

all_folds_preds.append(fold_2_preds)
print('All Folds Pred list after Fold 1+2 is \n\n{}\n\n'.format(all_folds_preds))


fold_3_preds = []

### Fold 3 ###
### All Batches: Total of 3 Predictions in the following format ###

fold_3_preds = [fold_3_pred_1, fold_3_pred_2]
fold_3_preds = np.concatenate(fold_3_preds, axis=0)
all_folds_preds.append(fold_3_preds)
print('All Folds Pred list after fold 1+2+3 is \n\n{}\n\n'.format(all_folds_preds))


fold_4_preds = []

### Fold 4 ###
### All Batches: Total of 3 Predictions in the following format ###

fold_4_preds = [fold_4_pred_1, fold_4_pred_2]
fold_4_preds = np.concatenate(fold_4_preds, axis=0)
all_folds_preds.append(fold_4_preds)
print('All Folds Pred list after fold 1+2+3+4 is \n\n{}\n\n'.format(all_folds_preds))


fold_5_preds = []

### Fold 5 ###
### All Batches: Total of 3 Predictions in the following format ###

fold_5_preds = [fold_5_pred_1, fold_5_pred_2]
fold_5_preds
fold_5_preds = np.concatenate(fold_5_preds, axis=0)
fold_5_preds
all_folds_preds.append(fold_5_preds)
print('All Folds Pred list after fold 1+2+3+4+5 is \n\n{}\n\n'.format(all_folds_preds))


###### Finally, we take np.mean over all_folds_preds over row wise calculation ######
###### Do not use concat here! ######
avg_pred_without_concat = np.mean(all_folds_preds, axis=0)
print('Average predictions is \n\n{}'.format(avg_pred_without_concat))

In [ ]:
# Converting to matrix in latex form

print((bmatrix(fold_1_preds)+'\n'))
print((bmatrix(fold_2_preds)+'\n'))
print((bmatrix(fold_3_preds)+'\n'))
print((bmatrix(fold_4_preds)+'\n'))
print((bmatrix(fold_5_preds)+'\n'))

five_folds_add = (fold_1_preds+fold_2_preds+fold_3_preds+fold_4_preds+fold_5_preds)
# five_folds_add
print((bmatrix(five_folds_add)+'\n'))

five_folds_add_avg = (fold_1_preds+fold_2_preds+fold_3_preds+fold_4_preds+fold_5_preds)/5
five_folds_add_avg
# print((bmatrix(five_folds_add_avg)+'\n'))

### Putting it all together


Final results: `all_folds_preds = [fold_1_preds, fold_2_preds, fold_3_preds, fold_4_preds, fold_5_preds]`

Finally, we average the `all_folds_preds` using `avg_preds = np.mean(all_folds_preds, axis=0)` to get our averaged predictions: 

    [[0.024  0.026  0.034  0.892  0.024 ]
     [0.032  0.824  0.03   0.044  0.07  ]
     [0.017  0.82   0.0281 0.0571 0.0778]]

Which is similar to our matrix:
    
$$\text{Dividing/Averaging all 5 Fold's Predictions}\begin{bmatrix}
  0.024 & 0.026 & 0.034 & 0.892 & 0.024\\
  0.032 & 0.824 & 0.03 & 0.044 & 0.07\\
  0.017 & 0.82 & 0.0281 & 0.0571 & 0.0778\\
\end{bmatrix}$$




**Numpy Shape Rows vs Columns**


There are 3 images in the test set, our final predictions for all 3 images across all 5 folds are presented below, with `shape` to be `[3,5]` which means 3 rows and 5 columns, where $\text{row}_i$ represents (1 by 5) vector containing 5 predictions in probabilities of how likely each class is; to put it even more explicit, if fold 1's prediction on the first image (call it $\text{image}_1$) is $[0.01, 0.02, 0.03, 0.9 , 0.04]$, then it means $\text{image}_1$ being class 1 is $1\%$, class 2 is $2\%$, class 3 is $3\%$, class 4 is $90\%$ and class 5, $4\%$.


One with Linear Algebra background can envision a (3 by 5) 2-dimensional array akin to a (3 x 5) **Matrix**.

So we are clear, the 5 matrices below represents the predictions of each fold. 


$$\text{Fold 1 Predictions}=\begin{bmatrix}
  0.01 & 0.02 & 0.03 & 0.9 & 0.04\\
  0.02 & 0.8 & 0.05 & 0.06 & 0.07\\
  0.01 & 0.8 & 0.03 & 0.09 & 0.07\\
\end{bmatrix}$$


$$\text{Fold 2 Predictions}\begin{bmatrix}
  0.03 & 0.05 & 0.01 & 0.88 & 0.03\\
  0.01 & 0.82 & 0.02 & 0.07 & 0.08\\
  0.005 & 0.81 & 0.0205 & 0.0555 & 0.109\\
\end{bmatrix}$$


$$\text{Fold 3 Predictions}=\begin{bmatrix}
  0.05 & 0.03 & 0.08 & 0.83 & 0.01\\
  0.05 & 0.89 & 0.01 & 0.02 & 0.03\\
  0.03 & 0.78 & 0.05 & 0.02 & 0.12\\
\end{bmatrix}$$

$$\text{Fold 4 Predictions}=\begin{bmatrix}
  0.02 & 0.01 & 0.03 & 0.92 & 0.02\\
  0.05 & 0.85 & 0.01 & 0.01 & 0.08\\
  0.02 & 0.88 & 0.03 & 0.05 & 0.02\\
\end{bmatrix}$$

$$\text{Fold 5 Predictions}\begin{bmatrix}
  0.01 & 0.02 & 0.02 & 0.93 & 0.02\\
  0.03 & 0.76 & 0.06 & 0.06 & 0.09\\
  0.02 & 0.83 & 0.01 & 0.07 & 0.07\\
\end{bmatrix}$$



All we are left to do is to add these 5 matrices, and divide by 5, as follows:

$$\text{Adding all 5 Fold's Predictions}\begin{bmatrix}
  0.12 & 0.13 & 0.17 & 4.46 & 0.12\\
  0.16 & 4.12 & 0.15 & 0.22 & 0.35\\
  0.085 & 4.1 & 0.1405 & 0.2855 & 0.389\\
\end{bmatrix}$$


Dividing by 5 (averaging):


$$\text{Dividing/Averaging all 5 Fold's Predictions}\begin{bmatrix}
  0.024 & 0.026 & 0.034 & 0.892 & 0.024\\
  0.032 & 0.824 & 0.03 & 0.044 & 0.07\\
  0.017 & 0.82 & 0.0281 & 0.0571 & 0.0778\\
\end{bmatrix}$$

## Submission Cautions

- When **COMMITTING YOUR NOTEBOOK**, do not print out anything in the `inference` function to debug if not it might take too much ram/gpu. 
- Check if you are using the same activation function, i.e. softmax vs sigmoid.
- Check in `inference` function if you are assigning the correct form of predictions to the test dataframe. In particular, refer to **SUBMISSION SAMPLE** to see what they want. If they want, say accuracy, then use `argmax()`, if just raw probabilities, then leave it as it is after you use sigmoid/softmax.

## Test-Time-Augmentation (TTA)

<a id="20"></a>

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#3aaa80; border-radius: 100px 100px; text-align:center">Saving and Loading Models' Weights</h1>

More often than not, we don't have resources to run the model day and night. Kaggle or Colab disconnects you once it reaches a certain number of hours. This bothers me when I just started out because my model has not **converged** yet.

Therefore, it is important to be able to **checkpoint** or **save** our model, for two main reasons:

**1. Save the model's weights and use it later to inference or make predictions.**

**2. Save the model's weights as a checkpoint and use it later to resume training.**

I will be using example from both the PyTorch website and a book that I bought. Please find below for references:

1. https://pytorch.org/tutorials/

2. Deep Learning with PyTorch - Step by Step A Beginner's Guide - Daniel Voigt

<a href="#top">Back to top</a>

### Question

Now, I am unsure if this has been answered, I remember my buddy telling me that anecdotally, one can "reset" the LR a little when one resumes training from a model's checkpoint.

It is a bit counter-intuitive to me, but say I trained a model for 16 epochs, with a custom scheduler, say OneCycleLr + Adam or something, then when you resume training, should we reset the initial learning rate? I would think resetting is counter-intuitive as the purpose of the learning rate is to tune it such that your model can slowly converge to the minima (global one if the function is convex).

So the question is:

Should I reset the learning rate when resume training, if yes, reset to what?

If we should not reset, what is a good way to "extract" the last learning rate, as some scheduler depends on factors like epochs…


Uncle CPMP's advice will add in and credit later.


In [ ]:
# def save_checkpoint(model, optimizer, scheduler, scaler, epoch, fold, seed, fname=fname):
#     checkpoint = {
#         'model': model.state_dict(),
#         'optimizer': optimizer.state_dict(),
#         'scheduler': scheduler.state_dict(),
#         'scaler': scaler.state_dict(),
#         'epoch': epoch,
#         'fold':fold,
#         'seed':seed,
#         }
#     torch.save(checkpoint, '../checkpoints/%s/%s_%d_%d.pt' % (fname, fname, fold, seed))

# def load_checkpoint(fold, seed, fname):
#     model = create_model().to(device)
#     optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=MAX_LR)
#     scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, 
#                                               pct_start=PCT_START, 
#                                               div_factor=DIV_FACTOR 
#                                               max_lr=MAX_LR, 
#                                               epochs=EPOCHS, 
#                                               steps_per_epoch=int(np.ceil(len(train_data_loader)/GRADIENT_ACCUMULATION)))
#     scaler = GradScaler()
#     checkpoint = torch.load('../checkpoints/%s/%s_%d_%d.pt' % (fname, fname, fold, seed))
#     model.load_state_dict(checkpoint['model'])
#     optimizer.load_state_dict(checkpoint['optimizer'])
#     scheduler.load_state_dict(checkpoint['scheduler'])
#     scaler.load_state_dict(checkpoint['scaler'])
#     return model, optimizer, scheduler, scaler, epoch



## What is a state_dict? 

In PyTorch, the learnable parameters (i.e. weights and biases) of an
``torch.nn.Module`` model are contained in the model’s *parameters*
(accessed with ``model.parameters()``). A *state_dict* is simply a
Python dictionary object that maps each layer to its parameter tensor.
Note that only layers with learnable parameters (convolutional layers,
linear layers, etc.) and registered buffers (batchnorm's running_mean)
have entries in the model’s *state_dict*. Optimizer
objects (``torch.optim``) also have a *state_dict*, which contains
information about the optimizer’s state, as well as the hyperparameters
used.

Because *state_dict* objects are Python dictionaries, they can be easily
saved, updated, altered, and restored, adding a great deal of modularity
to PyTorch models and optimizers.

## Simple Example

`in_channels` is the number of channels of the input to the convolutional layer. So, for example, in the case of the convolutional layer that applies to the image, `in_channels` refers to the number of channels of the image. In the case of an RGB image, `in_channels == 3` (red, green and blue); in the case of a gray image, `in_channels == 1`.

`out_channels` is the number of feature maps, which is often equivalent to the number of kernels that you apply to the input. See [here](https://stats.stackexchange.com/a/292064/82135) for more info. 

`kernel_size` is just the size of the kernel, usually `3x3` or `5x5`.

Convolutional Layer : Consider a convolutional layer which takes “l” feature maps as the input and has “k” feature maps as output. The filter size is $n*m$.

Here the input has l=32 feature maps as inputs, k=64 feature maps as outputs and filter size is n=3 and m=3. It is important to understand, that we don’t simply have a $3*3$ filter, but actually, we have $3*3*32$ filter, as our input has 32 dimensions. And as an output from first conv layer, we learn 64 different $3*3*32$ filters which total weights is $$n*m*k*l$$ Then there is a term called bias for each feature map. So, the total number of parameters are $$(n*m*l+1)*k$$

Think of the convolutional layer as a $nxn$ matrix, and $nxn = n^2$ is the weights (a.k.a what values to take when you apply the convolution.

In [ ]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels =6, kernel_size =5)
        self.pool = nn.MaxPool2d(2, 2)
        # here in channels = 6 because out channels of previous is 6.
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()

# Initialize optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
    


[Link to print parameters/weights for each layer](https://stackoverflow.com/questions/49201236/check-the-total-number-of-parameters-in-a-pytorch-model)

[Calculation of parameters in CNN](https://medium.com/@iamvarman/how-to-calculate-the-number-of-parameters-in-the-cnn-5bd55364d7ca)

[LeNet Architecture](https://engmrk.com/lenet-5-a-classic-cnn-architecture/#:~:text=The%20LeNet%2D5%20architecture%20consists,and%20finally%20a%20softmax%20classifier.)

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)

# indeed, 5x5x3x6 = 450, note the input here is 3 channels.

## Loading and Saving

**Save**

It is not a secret that one should use `torch.save` to save a model's weights in the following manner:

`torch.save(model.state_dict(), PATH)`



**Load**

And similarly, one should use `torch.load` to load a model's weights.

    model = TheModelClass(*args, **kwargs)
    model.load_state_dict(torch.load(PATH))
    model.eval()
 
 
**Advanced Save and Load**

One can also make what you save more sophisticated:

    torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                ...
                }, PATH)
                
                
And to load them, we do it as follows:

    model = TheModelClass(*args, **kwargs)
    optimizer = TheOptimizerClass(*args, **kwargs)

    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    model.eval()
    # - or -
    model.train()

**Caution**

Notice that the `load_state_dict()` function takes a dictionary object, **NOT a path to a saved object**. This means that you must deserialize the saved `state_dict` before you pass it to the `load_state_dict()` function. For example, you **CANNOT** load using `model.load_state_dict(PATH)`.

## **Loading and Saving on GPU**


**Save:**

      torch.save(model.state_dict(), PATH)

**Load:**



       device = torch.device("cuda")
       model = TheModelClass(*args, **kwargs)
       model.load_state_dict(torch.load(PATH))
       model.to(device)
 
When loading a model on a GPU that was trained and saved on GPU, simply
convert the initialized ``model`` to a CUDA optimized model using
``model.to(torch.device('cuda'))``. Also, be sure to use the
``.to(torch.device('cuda'))`` function on all model inputs to prepare
the data for the model. Note that calling ``my_tensor.to(device)``
returns a new copy of ``my_tensor`` on GPU. It does NOT overwrite
``my_tensor``. Therefore, remember to manually overwrite tensors:
``my_tensor = my_tensor.to(torch.device('cuda'))``. Make sure to call `input = input.to(device)` on any input tensors that you feed to the model, as you will see later.   

# Using it on a real example

**Reminder on what is a state_dict**

In PyTorch, the learnable parameters (i.e. weights and biases) of an
``torch.nn.Module`` model are contained in the model’s *parameters*
(accessed with ``model.parameters()``). A *state_dict* is simply a
Python dictionary object that maps each layer to its parameter tensor.
Note that only layers with learnable parameters (convolutional layers,
linear layers, etc.) and registered buffers (batchnorm's running_mean)
have entries in the model’s *state_dict*. Optimizer
objects (``torch.optim``) also have a *state_dict*, which contains
information about the optimizer’s state, as well as the hyperparameters
used.

In [ ]:
# # First, we define the model, since we will be loading our own "pretrained weights", there is no need
# # for one to set pretrained = True here, moreover, this competition is a No-Internet competition.

# effnet_model = CustomEfficientNet(config, pretrained=False)


# # The below code displays the parameter name, along with its size and tensors. 
# # Since the model is set to pretrained = False, the weights are initialized randomly (Xavier or something)
# # However, if you set pretrained=True, the weights are fixed because it is already pretrained.

# def count_parameters(model):
#     table = PrettyTable(["Modules", "Size of Modules", "Number of Parameters"])
#     total_params = 0
#     for param_name in model.state_dict():
#         param_tensor_size = model.state_dict()[param_name].size()
#         num_of_params = model.state_dict()[param_name].numel()
#         table.add_row([param_name, param_tensor_size, num_of_params])
#         total_params+=num_of_params
#     print(table)
#     print(f"Total Trainable Params: {total_params}")
#     return total_params
    
# count_parameters(effnet_model)

In [ ]:
# Secondly, we use torch.load to load the pretrained weights that we trained on. Make sure the weights were
# trained from the same EffficientNet, there was a time that I took a EfficientNetB3 weight and used it on
# EfficientNetB0 model. It will not prompt you an error and disaster ensues.

# state_dict_fold_1 = torch.load('../input/flowers/tf_efficientnet_b5_ns_fold_1_best_val.pt')

## CHECK EACH LAYER MATCHES IN SIZE.

In [ ]:
# # Thirdly, state_dict_fold_1 is an OrderedDict, before we proceed, it is very important to call
# # load_state_dict on the model! Because if not, we will get the same results as the previous 
# # count_parameters(effnet_model) when the model has yet to be trained!

# from collections import OrderedDict

# effnet_model.load_state_dict(state_dict_fold_1)
# # note that after you load the weights using load_state_dict, the effnet_model.state_dict changes!

# # Note that state_dict_fold_1 and effnet_model.state_dict() are equal!
# # Check the keys of both dict matches
# assert len(state_dict_fold_1) == len(effnet_model.state_dict())

# count = 0
# for param_name_1,param_name_2 in zip(state_dict_fold_1.keys(), effnet_model.state_dict().keys()):
#     if param_name_1 == param_name_2:
#         count+=1
# assert count==len(state_dict_fold_1) == len(effnet_model.state_dict())

# # ok the above code shows both have same param names, now to check if each tensor match:

# tensor_match_count = 0
# for param_name in state_dict_fold_1.keys() & effnet_model.state_dict().keys():
#     state_dict_fold_1_tensor = state_dict_fold_1[param_name].to(config.device)
#     effnet_model_state_dict = effnet_model.state_dict()[param_name].to(config.device)
#     if torch.all(torch.eq(state_dict_fold_1_tensor, effnet_model_state_dict)):
#         tensor_match_count+=1

# assert tensor_match_count==count==len(state_dict_fold_1) == len(effnet_model.state_dict())
# # so indeed all tensors matched.

[References](https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/175614)

[Understanding TTA](https://stepup.ai/test_time_data_augmentation/)

### PyTorch Flashcard Questions

We create the `inference_by_fold` function; a step by step explanation based on my own test set is as follows:

#### Qn 1: Why do we call `model.to(device)` in PyTorch?

- We equip the `model` to `device`, telling PyTorch whether we are using GPU/CPU;

#### Qn 2: Why do we set model to `eval()` mode in PyTorch during inference/validation?

- Subsequently, set `model` to `eval()` mode as we are in inference phase; This is extremely important because if you DO NOT set it `eval` mode, then your model predictions during inference will be off. Why? Well I can spend the whole day explaining, but the simple idea is in your model, there are many **regularization** methods like `nn.Dropout()` or common `nn.BatchNorm1d(2d)` layers. Then during inference, if your model mode is `train`, then your predictions will experience DROPOUT as well. This will lead to different, and possibly worse predictions every time you inference.

> What eval mode does to BatchNorm: During training, this layer keeps a running estimate of its computed mean and variance. The running sum is kept with a default momentum of 0.1. During evaluation, this running mean/variance is used for normalization.

#### Qn 3: Why do we `@torch.no_grad` in PyTorch? 

- [Answer](https://stackoverflow.com/questions/63351268/torch-no-grad-affects-on-model-accuracy): Since we are in inference phase, we say `torch.no_grad()` because we are no longer computing or storing gradients anymore.